# Spatial normalization of fluorescence to account for differences in illumination brightness

In [3]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

import pandas as pd

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

import time
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as patches
from scipy import stats
from scipy.spatial import distance
import numpy as np
from scipy import interpolate
from pathlib import Path
from sklearn.linear_model import LinearRegression
import matplotlib as mpl
from bsccm import BSCCM
from fluorescence_processing.shading_correction_util import *

data_root = '/home/henry/BSCCM_local/BSCCM/'
# data_root = '/home/henry/BSCCM_local/BSCCM-coherent/'

bsccm = BSCCM(data_root)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
bsccm.surface_marker_dataframe

,global_index,Fluor_690-_total_raw,Fluor_690-_background,Fluor_627-673_total_raw,Fluor_627-673_background,Fluor_585-625_total_raw,Fluor_585-625_background,Fluor_550-570_total_raw,Fluor_550-570_background,Fluor_500-550_total_raw,...,Fluor_500-550_shading,Fluor_500-550_shading_corrected,Fluor_550-570_shading,Fluor_550-570_shading_corrected,Fluor_585-625_shading,Fluor_585-625_shading_corrected,Fluor_627-673_shading,Fluor_627-673_shading_corrected,Fluor_690-_shading,Fluor_690-_shading_corrected
0,0,90.205154,74.643562,111.173241,92.166618,91.910187,77.581612,60.964123,58.214993,141.014313,...,80.666408,-9.546087,10.115647,0.244206,34.954990,4.599937,33.349353,17.730120,21.470533,14.561309
1,1,82.237915,75.487129,104.451164,94.646675,88.273483,79.380478,62.375282,60.246677,137.098679,...,78.718180,-17.419873,9.876530,-0.541537,33.993266,-1.870488,32.534823,7.757642,20.129025,5.836843
2,2,80.798347,74.069023,103.424004,92.087128,87.543541,77.186424,59.857498,57.525318,136.572571,...,78.713359,-12.872697,9.765018,-0.042141,34.339840,0.468060,32.555925,9.668359,20.685243,5.601074
3,3,79.028641,74.433380,92.806908,83.702690,78.677719,69.564919,46.312710,44.212730,120.823174,...,86.203365,-7.148947,10.419661,-3.929642,36.784052,-1.453486,35.674436,5.704854,21.041925,6.570466
4,4,82.644333,77.617538,98.803795,88.936920,83.008171,73.497032,52.991127,50.805092,129.006073,...,82.367954,-12.806695,10.256053,-4.428715,36.088059,-1.035731,35.341019,8.155553,21.453345,8.231029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412936,412936,74.466263,73.023483,75.277222,69.873268,63.614056,57.465630,51.774380,50.261669,98.392014,...,80.195842,3.854952,9.851740,0.801560,35.055580,5.014530,33.796833,2.777335,21.767867,3.376433
412937,412937,75.483070,73.832535,76.499336,71.274117,64.562302,58.331261,56.651958,55.158417,99.938904,...,76.685799,1.525111,9.814024,0.693378,33.739942,4.546722,32.451605,2.060166,20.287460,3.327853
412938,412938,72.927307,71.721359,72.462761,68.135788,60.908398,56.115700,51.587204,50.231682,93.220558,...,79.952670,0.479350,9.736240,0.633300,34.247302,3.530663,32.423705,1.562294,19.844133,3.586604
412939,412939,70.676392,69.221497,78.399567,72.564072,61.523308,55.135784,56.591564,54.959549,100.411942,...,82.330931,2.160017,10.218266,1.706385,35.313222,0.771515,34.760057,2.876208,22.267203,-2.612637


## Plan
 - Visualize current state
 - Compute median of background measurements to get slide specific, channel_specific background
 - Compute median of cells to get slide specific, channel_specific, foreground
 - Compute Loess of each one to get a continuous measurement
 - Subtract background from foregound and normalize to get shading correction
 - final result: foreground minus background, divide by normalized shading, visualize result


## Visualize spatial histograms of raw brightness

In [39]:
%matplotlib widget

import matplotlib.pyplot as plt

channel_names = [
    'Fluor_426-446_total_raw',
    'Fluor_500-550_total_raw',
    'Fluor_550-570_total_raw',
    'Fluor_585-625_total_raw',
    'Fluor_627-673_total_raw',
    'Fluor_690-_total_raw',
    ]

show_spatial_histograms(bsccm, channel_names, figsize=(9, 8.5), batch=0, N=20)
plt.savefig('/home/henry/leukosight_data/figures/demixing/raw_brightness_histograms0.pdf', 
            transparent=True)

show_spatial_histograms(bsccm, channel_names, figsize=(9, 8.5), batch=1, N=20)
plt.savefig('/home/henry/leukosight_data/figures/demixing/raw_brightness_histograms1.pdf', 
            transparent=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Visualize spatial histograms of backgrounds

In [42]:
background_names = ['Fluor_426-446_background',
 'Fluor_500-550_background',
 'Fluor_550-570_background',
 'Fluor_585-625_background',
 'Fluor_627-673_background',
 'Fluor_690-_background']

show_spatial_histograms(bsccm, background_names, figsize=(9, 8.5), batch=0, N=20)
plt.savefig('/home/henry/leukosight_data/figures/demixing/raw_brightness_backgrounds0.pdf', 
            transparent=True)

show_spatial_histograms(bsccm, background_names, figsize=(9, 8.5), batch=1, N=20)
plt.savefig('/home/henry/leukosight_data/figures/demixing/raw_brightness_backgrounds1.pdf', 
            transparent=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Visualize computed smooth function of background intensity pattern

In [67]:
replicate = 0
antibodies_list = bsccm.index_dataframe['antibodies'].unique()

background_names = ['Fluor_426-446_background',
 'Fluor_500-550_background',
 'Fluor_550-570_background',
 'Fluor_585-625_background',
 'Fluor_627-673_background',
 'Fluor_690-_background']

for batch in bsccm.index_dataframe['batch'].unique():
    fig, ax = plt.subplots(len(antibodies_list), len(background_names), figsize=(9, 8.5))
    
    for i, antibodies in enumerate(antibodies_list):
        print('{} {}\r'.format(batch, antibodies))
        mask = np.logical_and(np.logical_and(bsccm.index_dataframe['antibodies'] == antibodies,
                                            bsccm.index_dataframe['batch'] == batch), 
                                            bsccm.index_dataframe['slide_replicate'] == replicate)
        mask_indices = np.flatnonzero(mask)

        for j, channel in enumerate(background_names):
            if j == 0:
                ax[i, j].set_ylabel(antibodies)
            y_pos = bsccm.index_dataframe.loc[mask_indices, 'position_in_fov_y_pix'].to_numpy()
            x_pos = bsccm.index_dataframe.loc[mask_indices, 'position_in_fov_x_pix'].to_numpy()
            fluor = bsccm.surface_marker_dataframe.loc[mask_indices, background_names[j]].to_numpy()
            
            plot_gridded_lowess(y_pos, x_pos, fluor, ax[i, j], alpha=1000 / x_pos.shape[0] )
#             plot_gridded_lowess(y_pos, x_pos, fluor, ax[i, j], alpha=100 / x_pos.shape[0] )
    fig.suptitle('Batch {}'.format(batch))
    plt.savefig('/home/henry/leukosight_data/figures/demixing/brightness_backgrounds_lowess{}.pdf'.format(batch), 
            transparent=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1 CD45
1 CD123
1 unstained
1 CD19
1 CD56
1 all
1 CD14
1 CD16
1 HLA-DR
1 CD3


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0 CD45
0 CD123
0 unstained
0 CD19
0 CD56
0 all
0 CD14
0 CD16
0 HLA-DR
0 CD3


### 1) Go through all slides, compute the background for each, and store lowess background subtracted values
This one takes a long time to run

In [9]:
background_names = ['Fluor_426-446_background',
 'Fluor_500-550_background',
 'Fluor_550-570_background',
 'Fluor_585-625_background',
 'Fluor_627-673_background',
 'Fluor_690-_background']


antibodies_list = bsccm.index_dataframe['antibodies'].unique()

for batch in range(2):
    for i, antibodies in enumerate(antibodies_list):
        print(antibodies)
        for replicate in range(2):
            mask = np.logical_and(np.logical_and(bsccm.index_dataframe['antibodies'] == antibodies,
                                            bsccm.index_dataframe['batch'] == batch), 
                                            bsccm.index_dataframe['slide_replicate'] == replicate)
            mask_indices = np.flatnonzero(mask)
            if mask_indices.size == 0:
                continue

            for j, channel in enumerate(background_names):            
                y_pos = bsccm.index_dataframe.loc[mask_indices, 'position_in_fov_y_pix'].to_numpy()
                x_pos = bsccm.index_dataframe.loc[mask_indices, 'position_in_fov_x_pix'].to_numpy()
                fluor = bsccm.surface_marker_dataframe.loc[mask_indices, channel].to_numpy()
                
                yx_query_points = np.stack([y_pos, x_pos], axis=-1)

                # compute alpha adaptively so it corresponds to same number cells
#                 alpha = 100 / x_pos.shape[0] 
                alpha = 1000 / x_pos.shape[0]
                predictions = lowess_2d(y_pos, x_pos, fluor, 
                          yx_query_points, alpha=alpha, weight_fn='tricubic')
                bsccm.surface_marker_dataframe.loc[mask_indices, channel + '_lowess'] = predictions            
                

bsccm.surface_marker_dataframe.to_csv(data_root + 'BSCCM_surface_markers.csv', index=False)

CD45
CD123
unstained
CD19
CD56
all
CD14
CD16
HLA-DR
CD3
CD45
CD123
unstained
CD19
CD56
all
CD14
CD16
HLA-DR
CD3


### 2) Compute final normalized fluorescence

Pool over markers (subset of 1000 from each slide?), and compute LOWESS over background-subtracted cells divided by mean intensity

    a) Visualize the computed LOWESS
    b) Use it to compute final normalized fluorescence
    
Output: normalized fluorescence


Compile a bunch of measurements of background subtracted brightness for the purposes of creating a shading correction
Since the brightest cells will give the most accurate signal for this, do a spatially local thresholding to only include 
the top 20% brightness cells for each location in the slide

In [69]:
background_names_lowess = ['Fluor_426-446_background_lowess',
 'Fluor_500-550_background_lowess',
 'Fluor_550-570_background_lowess',
 'Fluor_585-625_background_lowess',
 'Fluor_627-673_background_lowess',
 'Fluor_690-_background_lowess']

channel_names = [
    'Fluor_426-446',
    'Fluor_500-550',
    'Fluor_550-570',
    'Fluor_585-625',
    'Fluor_627-673',
    'Fluor_690-',
    ]

antibodies_list = bsccm.index_dataframe['antibodies'].unique()

data_by_channel = {c: {'x_pos': [],
                     'y_pos': [],
                     'background_sub_fluor': [],
                       'raw_fluor_of_brightest': [],
                      } for c in channel_names}

for batch in bsccm.index_dataframe['batch'].unique():
    for i, antibodies in enumerate(antibodies_list):

        if antibodies == 'unstained':
            continue
            #Don't use unstained because their fluorescence is low so they give inaccurate measurements
            
        print(antibodies)
        for replicate in range(2):
            if antibodies == 'unstained' and batch == 1 and replicate == 1:
                #This one has the fieldstop positioned slightly differently and so is not to be trusted
                continue
            mask = np.logical_and(np.logical_and(bsccm.index_dataframe['antibodies'] == antibodies,
                                            bsccm.index_dataframe['batch'] == batch), 
                                            bsccm.index_dataframe['slide_replicate'] == replicate)
            mask_indices = np.flatnonzero(mask)
            if mask_indices.size == 0:
                continue
                

            for ch_index in range(len(background_names_lowess)):
                y_pos = bsccm.index_dataframe.loc[mask_indices, 'position_in_fov_y_pix'].to_numpy()
                x_pos = bsccm.index_dataframe.loc[mask_indices, 'position_in_fov_x_pix'].to_numpy()
                fluor_foreground = bsccm.surface_marker_dataframe.loc[mask_indices, 
                                                           channel_names[ch_index] + '_total_raw'].to_numpy()
                fluor_background_lowess = bsccm.surface_marker_dataframe.loc[mask_indices, 
                                                           background_names_lowess[ch_index]].to_numpy()
                fluor_background_subtracted = fluor_foreground - fluor_background_lowess
                
                # apply spatial threshold: only look at top 10% of brightest cells at each spatial position
                computed = stats.binned_statistic_2d(y_pos, x_pos, fluor_background_subtracted, bins=20, 
                                                             statistic= lambda x: np.percentile(x, 90),
                                                     expand_binnumbers=True
                                                    )
                spatial_threshold_mask = [f > computed.statistic[computed.binnumber[0, i] - 1,
                                                                 computed.binnumber[1, i] - 1] for i, f in enumerate(fluor_background_subtracted)]
                fluor_background_subtracted = fluor_background_subtracted[spatial_threshold_mask]
                fluor_foreground = fluor_foreground[spatial_threshold_mask]
                x_pos = x_pos[spatial_threshold_mask]
                y_pos = y_pos[spatial_threshold_mask]
                
                # limit to 2000 per slide to make comptuation easier
                if x_pos.size > 2000:
                    selected_indices = np.random.choice(np.arange(x_pos.size), 2000, replace=False)
                    fluor_background_subtracted = fluor_background_subtracted[selected_indices]
                    fluor_foreground = fluor_foreground[selected_indices]
                    x_pos = x_pos[selected_indices]
                    y_pos = y_pos[selected_indices]
                
                
                data_by_channel[channel_names[ch_index]]['x_pos'].append(x_pos)
                data_by_channel[channel_names[ch_index]]['y_pos'].append(y_pos)
                data_by_channel[channel_names[ch_index]]['background_sub_fluor'].append(fluor_background_subtracted) 
                data_by_channel[channel_names[ch_index]]['raw_fluor_of_brightest'].append(fluor_foreground) 


for channel in channel_names:
    data_by_channel[channel]['x_pos'] = np.concatenate(data_by_channel[channel]['x_pos'])
    data_by_channel[channel]['y_pos'] = np.concatenate(data_by_channel[channel]['y_pos'])
    data_by_channel[channel]['background_sub_fluor'] = np.concatenate(data_by_channel[channel]['background_sub_fluor'])
    data_by_channel[channel]['raw_fluor_of_brightest'] = np.concatenate(data_by_channel[channel]['raw_fluor_of_brightest'])


CD45
CD123
CD19
CD56
all
CD14
CD16
HLA-DR
CD3
CD45
CD123
CD19
CD56
all
CD14
CD16
HLA-DR
CD3


### Visualize spatial histograms of background subtracted

In [74]:
fig, ax = plt.subplots(1, 6, figsize=(10,1))

N = 20

for c_index, channel in enumerate(channel_names):
    print(channel)
    y_pos = data_by_channel[channel]['y_pos']
    x_pos = data_by_channel[channel]['x_pos']
    
    non_background_sub_fluor = data_by_channel[channel]['raw_fluor_of_brightest']
    
    stat = stats.binned_statistic_2d(y_pos, x_pos, non_background_sub_fluor, bins=N, 
                                         statistic= 'mean',
                                         expand_binnumbers=True
                                        ).statistic
    
    contrast_max = np.nanpercentile(stat, 95)
    contrast_min = np.nanpercentile(stat, 5)
    im = ax[c_index].imshow(stat, cmap='inferno', vmin= contrast_min, vmax=contrast_max)
    ax[c_index].set_axis_off()
    plt.colorbar(im, ax=ax[c_index], fraction=0.1, aspect=8)
    ax[c_index].set_title(channel + '_non_background_subtracted')
    

plt.savefig('/home/henry/leukosight_data/figures/demixing/brightest_cells_distribution.pdf', 
            transparent=True)

fig, ax = plt.subplots(1, 6, figsize=(10,1))

for c_index, channel in enumerate(channel_names):
    print(channel)
    y_pos = data_by_channel[channel]['y_pos']
    x_pos = data_by_channel[channel]['x_pos']
    
    fluor_foreground = bsccm.surface_marker_dataframe.loc[
        selected_indices, channel_names[ch_index] + '_total_raw'].to_numpy()
  
    fluor = data_by_channel[channel]['background_sub_fluor']    
    
    stat = stats.binned_statistic_2d(y_pos, x_pos, fluor, bins=N, 
                                         statistic= 'mean',
                                         expand_binnumbers=True
                                        ).statistic
    
    contrast_max = np.nanpercentile(stat, 95)
    contrast_min = np.nanpercentile(stat, 5)
    im = ax[c_index].imshow(stat, cmap='inferno', vmin= contrast_min, vmax=contrast_max)
    ax[c_index].set_axis_off()
    plt.colorbar(im, ax=ax[c_index], fraction=0.1, aspect=8)
    ax[c_index].set_title(channel)

plt.savefig('/home/henry/leukosight_data/figures/demixing/brightest_cells_background_subtracted.pdf', 
            transparent=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Fluor_426-446
Fluor_500-550
Fluor_550-570
Fluor_585-625
Fluor_627-673
Fluor_690-


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Fluor_426-446
Fluor_500-550
Fluor_550-570
Fluor_585-625
Fluor_627-673
Fluor_690-


### Visualize LOWESS
This is the factor you'll divide by to do shadign correction

In [68]:
#Normalize data before computing LOWESS
fig, ax = plt.subplots(1, 6, figsize=(10, 1))

N = 20

for c_index, channel in enumerate(channel_names):
    print(channel)
    y_pos = data_by_channel[channel]['y_pos']
    x_pos = data_by_channel[channel]['x_pos']
    
    fluor = data_by_channel[channel]['background_sub_fluor']
    
    # normalize 
    fluor = fluor / np.mean(fluor)
    
    alpha = 1000 / x_pos.shape[0] 
#     alpha = 100 / x_pos.shape[0] 
    plot_gridded_lowess(y_pos, x_pos, fluor, ax[c_index], N=N, weight_fn='tricubic', alpha=alpha)
    ax[c_index].set_title(channel)
    ax[c_index].spines['right'].set_visible(False)
    ax[c_index].spines['bottom'].set_visible(False)
    ax[c_index].spines['top'].set_visible(False)
    ax[c_index].spines['left'].set_visible(False)
plt.savefig('/home/henry/leukosight_data/figures/demixing/shading.pdf', 
            transparent=True)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Fluor_426-446
Fluor_500-550
Fluor_550-570
Fluor_585-625
Fluor_627-673
Fluor_690-


### Use normalized LOWESS to do shading correction

In [28]:
channel_names = [
    'Fluor_426-446',
    'Fluor_500-550',
    'Fluor_550-570',
    'Fluor_585-625',
    'Fluor_627-673',
    'Fluor_690-',
    ]

background_names_lowess = ['Fluor_426-446_background_lowess',
 'Fluor_500-550_background_lowess',
 'Fluor_550-570_background_lowess',
 'Fluor_585-625_background_lowess',
 'Fluor_627-673_background_lowess',
 'Fluor_690-_background_lowess']

for i in range(6):            
    print(channel_names[i])
    
    #Do one marker at a time to save on memory
    batch_size = 1000
    for chunk_index in range(bsccm.index_dataframe.global_index.size // batch_size + 1):
        print('{} of {}    \r'.format(chunk_index, bsccm.index_dataframe.global_index.size // batch_size + 1), end='')
        mask = np.logical_and(bsccm.index_dataframe.global_index >= chunk_index * batch_size,
                              bsccm.index_dataframe.global_index < (chunk_index + 1) * batch_size)
        
        indices = np.flatnonzero(mask)
        
        
        # 1) Load background subtracted
        y_pos = bsccm.index_dataframe.loc[indices, 'position_in_fov_y_pix'].to_numpy()
        x_pos = bsccm.index_dataframe.loc[indices, 'position_in_fov_x_pix'].to_numpy()
        foreground = bsccm.surface_marker_dataframe.loc[indices, channel_names[i] + '_total_raw'].to_numpy()
        background = bsccm.surface_marker_dataframe.loc[indices, background_names_lowess[i]].to_numpy()

        background_corrected = foreground - background

        #a subset of the data in that channel, so whole thing doesnt take forever
        channel_ref_examples = data_by_channel[channel_names[i]]['background_sub_fluor']
        channel_ref_y_pos = data_by_channel[channel]['y_pos']
        channel_ref_x_pos = data_by_channel[channel]['x_pos']

        #TODO:
        # 2a) Compute LOWESS on normalized data

        yx_query_points = np.stack([y_pos, x_pos], axis=-1)


        # compute the shading -- lowess over a representative set of background examples
        alpha = 1000 / channel_ref_x_pos.shape[0] 
#         alpha = 100 / channel_ref_x_pos.shape[0] 
        shading = lowess_2d(channel_ref_y_pos, channel_ref_x_pos, channel_ref_examples, 
                  yx_query_points, alpha=alpha, weight_fn='tricubic')
        bsccm.surface_marker_dataframe.loc[indices, channel_names[i] + '_shading'] = shading
        
    #compute normalized version for all at once
    shading = bsccm.surface_marker_dataframe[channel_names[i] + '_shading'].to_numpy()
    #normalize to avoid numerical errors
    shading = shading / np.mean(shading)

    # 2b) Divide by shading correction factor
    foreground = bsccm.surface_marker_dataframe[channel_names[i] + '_total_raw']
    background = bsccm.surface_marker_dataframe[background_names_lowess[i]]

    corrected = (foreground - background) / shading

    # 3) Store result
    bsccm.surface_marker_dataframe[channel_names[i] + '_shading_corrected'] = corrected

#Resave
bsccm.surface_marker_dataframe.to_csv(data_root + 'BSCCM_surface_markers.csv', index=False)

Fluor_426-446


KeyboardInterrupt: 

In [159]:
bsccm.surface_marker_dataframe

,global_index,Fluor_690-_total_raw,Fluor_690-_background,Fluor_627-673_total_raw,Fluor_627-673_background,Fluor_585-625_total_raw,Fluor_585-625_background,Fluor_550-570_total_raw,Fluor_550-570_background,Fluor_500-550_total_raw,...,Fluor_500-550_shading,Fluor_500-550_shading_corrected,Fluor_550-570_shading,Fluor_550-570_shading_corrected,Fluor_585-625_shading,Fluor_585-625_shading_corrected,Fluor_627-673_shading,Fluor_627-673_shading_corrected,Fluor_690-_shading,Fluor_690-_shading_corrected
0,0,90.205154,74.643562,111.173241,92.166618,91.910187,77.581612,60.964123,58.214993,141.014313,...,80.666408,-9.546087,10.115647,0.244206,34.954990,4.599937,33.349353,17.730120,21.470533,14.561309
1,1,82.237915,75.487129,104.451164,94.646675,88.273483,79.380478,62.375282,60.246677,137.098679,...,78.718180,-17.419873,9.876530,-0.541537,33.993266,-1.870488,32.534823,7.757642,20.129025,5.836843
2,2,80.798347,74.069023,103.424004,92.087128,87.543541,77.186424,59.857498,57.525318,136.572571,...,78.713359,-12.872697,9.765018,-0.042141,34.339840,0.468060,32.555925,9.668359,20.685243,5.601074
3,3,79.028641,74.433380,92.806908,83.702690,78.677719,69.564919,46.312710,44.212730,120.823174,...,86.203365,-7.148947,10.419661,-3.929642,36.784052,-1.453486,35.674436,5.704854,21.041925,6.570466
4,4,82.644333,77.617538,98.803795,88.936920,83.008171,73.497032,52.991127,50.805092,129.006073,...,82.367954,-12.806695,10.256053,-4.428715,36.088059,-1.035731,35.341019,8.155553,21.453345,8.231029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412936,412936,74.466263,73.023483,75.277222,69.873268,63.614056,57.465630,51.774380,50.261669,98.392014,...,80.195842,3.854952,9.851740,0.801560,35.055580,5.014530,33.796833,2.777335,21.767867,3.376433
412937,412937,75.483070,73.832535,76.499336,71.274117,64.562302,58.331261,56.651958,55.158417,99.938904,...,76.685799,1.525111,9.814024,0.693378,33.739942,4.546722,32.451605,2.060166,20.287460,3.327853
412938,412938,72.927307,71.721359,72.462761,68.135788,60.908398,56.115700,51.587204,50.231682,93.220558,...,79.952670,0.479350,9.736240,0.633300,34.247302,3.530663,32.423705,1.562294,19.844133,3.586604
412939,412939,70.676392,69.221497,78.399567,72.564072,61.523308,55.135784,56.591564,54.959549,100.411942,...,82.330931,2.160017,10.218266,1.706385,35.313222,0.771515,34.760057,2.876208,22.267203,-2.612637


### Validate the final result by looking at spatial histograms of normalized fluorescence
Which should no longer have an obvious spatial distribution

In [66]:
corrected_channel_names = [
    'Fluor_426-446_shading_corrected',
    'Fluor_500-550_shading_corrected',
    'Fluor_550-570_shading_corrected',
    'Fluor_585-625_shading_corrected',
    'Fluor_627-673_shading_corrected',
    'Fluor_690-_shading_corrected',
    ]

show_spatial_histograms(bsccm, corrected_channel_names, N=20, figsize=(9, 8.5), batch=0)

plt.savefig('/home/henry/leukosight_data/figures/demixing/shading_corrected_brightness0.pdf', 
            transparent=True)

show_spatial_histograms(bsccm, corrected_channel_names, N=20, figsize=(9, 8.5), batch=1)

plt.savefig('/home/henry/leukosight_data/figures/demixing/shading_corrected_brightness1.pdf', 
            transparent=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …